# 이 미친 인간들이 과시 데이터를 전부 링크로 퉁치면 어떡해

In [27]:
links = ['https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1872.0000-20101008.B043a_065_00633_XXX',
'https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=holder&upPath=01%5E0167%5E016715%5EG002%2BAKS%2BKSM-XD.1555.1111-20101008.B001a_001_00143_XXX&dataId=G002%2BAKS%2BKSM-XD.1555.1111-20101008.B001a_001_00143_XXX',
'https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1873.0000-20101008.B047a_071_00188_XXX']

## 안내정보와 상세정보만 있는 경우

In [32]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 불러오기
df = pd.read_csv("raw.csv")

# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 결과 저장용
original_list = []
translate_list = []
info_list = []
error_log = []

# 크롤링 루프
for i, url in enumerate(df['webResourceURL']):
    print(f"[{i+1}/{len(df)}] 처리 중: {url}")

    # URL 유효성 검사
    if pd.isna(url) or not isinstance(url, str) or not url.startswith("http"):
        original_list.append("")
        translate_list.append("")
        info_list.append("")
        error_log.append({"index": i, "url": url, "error": "invalid or missing URL"})
        continue

    try:
        driver.get(url)
        time.sleep(3)

        # 원문
        try:
            original = driver.find_element(By.XPATH, '//div[@class="org_text text_body_scroll text_body_wrap scroll_area"]')
            original_list.append(original.text.strip())
        except:
            original_list.append("")

        # 해설문
        try:
            translate = driver.find_element(By.XPATH, '//div[@class="text_body_scroll text_body_wrap scroll_area"]')
            translate_list.append(translate.text.strip())
        except:
            translate_list.append("")

        # 안내정보/상세정보
        try:
            info = driver.find_element(By.XPATH, '//div[@class="text__body--info__text"]')
            info_list.append(info.text.strip())
        except:
            info_list.append("")

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        original_list.append("")
        translate_list.append("")
        info_list.append("")
        error_log.append({"index": i, "url": url, "error": str(e)})

# 드라이버 종료
driver.quit()

# 컬럼 추가
df['original'] = original_list
df['translate'] = translate_list
df['info'] = info_list

# 결과 저장
df.to_csv("시권_본문_3컬럼_추출결과.csv", index=False, encoding='utf-8-sig')

# 에러 로그 저장
error_df = pd.DataFrame(error_log)
error_df.to_csv("시권_크롤링_에러로그.csv", index=False, encoding='utf-8-sig')

print("\n✅ 3개 영역 본문 추출 완료 및 에러 로그 저장!")

[1/513] 처리 중: http://www.emuseum.go.kr/detail?relicId=PS0100200100106199100000
[2/513] 처리 중: nan
[3/513] 처리 중: https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1555.0000-20101008.B001a_001_00140_XXX
[4/513] 처리 중: https://archive.aks.ac.kr/link.do?dataUCI=G002+AKS+KSM-XD.1555.1111-20101008.B001a_001_00143_XXX
[5/513] 처리 중: https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1561.0000-20101008.B028a_043_00397_XXX
[6/513] 처리 중: https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1582.0000-20101008.B028a_043_00398_XXX
[7/513] 처리 중: https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1588.0000-20101008.B021a_033_00485_XXX
[8/513] 처리 중: https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPa

## 원문정보만 있는 경우

- 따로 뽑을 만 한 게 없는 거 같은데 공란

## 원문텍스트와 번역문이 있다면

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창을 띄우지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 접근할 URL
driver.get(links[1])
time.sleep(3)  # JavaScript 로딩 대기

# 본문 영역 추출 시도
try:
    elements = driver.find_elements(By.XPATH, """""")
    for i, elem in enumerate(elements):
        print(f"[{i}] {elem.text}")
except Exception as e:
    print("❌ 요소 추출 실패:", e)

driver.quit()

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 불러오기
df = pd.read_csv("raw.csv")

# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 결과 저장용
extracted_texts = []
error_log = []

# 크롤링 루프
for i, url in enumerate(df['webResourceURL']):
    print(f"[{i+1}/{len(df)}] 처리 중: {url}")

    if pd.isna(url) or not isinstance(url, str) or not url.startswith("http"):
        extracted_texts.append("")
        error_log.append({"index": i, "url": url, "error": "invalid or missing URL"})
        continue

    try:
        driver.get(url)
        time.sleep(3)
        element = driver.find_element(By.XPATH, '')
        extracted_texts.append(element.text.strip())
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        extracted_texts.append("")
        error_log.append({"index": i, "url": url, "error": str(e)})

# 드라이버 종료
driver.quit()

# 본문 열 추가
df['본문'] = extracted_texts

# 결과 저장
df.to_csv("시권_본문_추출결과.csv", index=False, encoding='utf-8-sig')

# 에러 로그 저장
error_df = pd.DataFrame(error_log)
error_df.to_csv("시권_크롤링_에러로그.csv", index=False, encoding='utf-8-sig')

print("\n✅ 본문 추출 완료 및 에러 로그 저장!")

[1/513] 처리 중: http://www.emuseum.go.kr/detail?relicId=PS0100200100106199100000


KeyboardInterrupt: 

In [ ]:
prin